<a href="https://colab.research.google.com/github/kmouleejunhyuk/bigdata_analysist_practical/blob/main/baseline_big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#x_train
#   Column   Non-Null Count  Dtype  
# ---  ------   --------------  -----  
#  0   cust_id  3500 non-null   int64  
#  1   총구매액     3500 non-null   int64  
#  2   최대구매액    3500 non-null   int64  
#  3   환불금액     1205 non-null   float64
#  4   주구매상품    3500 non-null   object 
#  5   주구매지점    3500 non-null   object 
#  6   내점일수     3500 non-null   int64  
#  7   내점당구매건수  3500 non-null   float64
#  8   주말방문비율   3500 non-null   float64
#  9   구매주기     3500 non-null   int64  

#y_train
# Data columns (total 2 columns):
#  #   Column   Non-Null Count  Dtype
# ---  ------   --------------  -----
#  0   cust_id  3500 non-null   int64
#  1   gender   3500 non-null   int64

In [ ]:
import time
st = time.time()
import pandas as pd
import numpy as np

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

ROOT = '/content/drive/MyDrive/big_2021/data/official/type2/'
enc = 'cp949'
x_train = pd.read_csv(ROOT + 'X_train.csv', encoding = enc)
y_train = pd.read_csv(ROOT + 'y_train.csv', encoding = enc)
x_test = pd.read_csv(ROOT + 'X_test.csv', encoding = enc)

import sklearn 
import scipy.stats

def view_df(df: pd.DataFrame, view: bool)-> None:
	if view == True:
		print(df.info())
		print(df.describe().transpose().to_string())
		print(df.corr().to_string())
		print(df.isnull().sum())
		
	return None

test_id = x_test['cust_id']
x_train.drop(['cust_id'], axis = 1, inplace = True)
y_train.drop(['cust_id'], axis = 1, inplace = True)
x_test.drop(['cust_id'], axis = 1, inplace = True)

raw = pd.concat([x_train, y_train], axis = 1)

VIEW = False
view_df(raw, VIEW)

#환불금액 nan 0으로 치환
raw['환불금액'].replace({np.nan: 0}, inplace = True)
x_test['환불금액'].replace({np.nan: 0}, inplace = True)

#구매금액을 5개 그룹으로 나누어라
hist, bins = np.histogram(raw['총구매액'], bins = 5)

bins[0] = -1 * np.inf
bins[-1] = np.inf
raw['구매금액_cut'] = pd.cut(raw['총구매액'], bins, include_lowest = True, labels = [0, 1, 2, 3, 4])
x_test['구매금액_cut'] = pd.cut(x_test['총구매액'], bins, include_lowest = True, labels = [0, 1, 2, 3, 4])

raw['구매금액_cut'] = raw['구매금액_cut'].astype(int)
x_test['구매금액_cut'] = x_test['구매금액_cut'].astype(int)

#labelencoding
from sklearn.preprocessing import LabelEncoder
le_m = LabelEncoder()
le_p = LabelEncoder()

cat_cols = [x for x in raw.columns if raw[x].dtype == 'object']
tmp = pd.concat([raw[cat_cols].copy(), x_test[cat_cols].copy()], axis = 0)
le_m.fit(tmp['주구매상품'])
le_p.fit(tmp['주구매지점'])

raw['주구매상품'] = le_m.transform(raw['주구매상품'])
raw['주구매지점'] = le_p.transform(raw['주구매지점'])
x_test['주구매상품'] = le_m.transform(x_test['주구매상품'])
x_test['주구매지점'] = le_p.transform(x_test['주구매지점'])

#이상치 제거(zscore)
from scipy.stats import zscore
num_cols = [x for x in raw.columns if raw[x].dtype != 'object']
raw = raw[(np.abs(zscore(raw[num_cols])) < 3).all(axis = 1)]

#학습
x_train = raw[[x for x in raw.columns if x != 'gender']]
y_train = raw['gender']

from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

model = RandomForestClassifier()
scaler = StandardScaler()

pipe = Pipeline(steps=[('scaler', scaler), ('model', model)])

score = cross_val_score(pipe, x_train, y_train, cv = 5, scoring = 'roc_auc')
print(score.mean())

pipe.fit(x_train, y_train)
res = pipe.predict_proba(x_test)[:, 1]
print(res)

sub = pd.DataFrame()
sub['custid'] = test_id
sub['gender'] = res

sub.to_csv('data/002001685.csv', index = False)


print('time: ', time.time() - st)


In [ ]:
raw_sangpoom.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])